# 🤖 Pro Investment Analyst Swarm (Gemini 1.5 Flash)

This robust workflow generates a 6-section Investment Memo without hitting API Rate Limits.

### 🏗️ Architecture
1.  **InitialSearchAgent**: Finds the PDF URL.
2.  **FileAnalystAgent**: Downloads PDF -> Extracts 6 key sections -> Compiles **ONE** Master Report.
3.  **GapClosureLoop**: Reviews the report -> Fills gaps -> Approves.

### ⚡ Rate Limit Protection
- Uses **Gemini 1.5 Flash** (Higher 15 RPM limit).
- **Auto-Sleep:** The tool forces a 5s pause between queries.
- **Cooldown:** System pauses for 15s between Agents.

In [ ]:
# @title 1. Setup & Authentication
!pip install -U -q google-adk

import os
import time
import requests
import re
import json
from google import genai
from google.genai import types
from google.adk.agents import LlmAgent, SequentialAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool, google_search, ToolContext
from google.adk.agents.callback_context import CallbackContext
from kaggle_secrets import UserSecretsClient

# --- CONFIGURATION ---
MODEL_NAME = "gemini-1.5-flash"  # STABLE MODEL (15 RPM Limit)

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
except:
    # os.environ["GOOGLE_API_KEY"] = "PASTE_KEY_HERE_IF_LOCAL"
    pass

if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError("❌ API Key missing! Add 'GOOGLE_API_KEY' to Secrets.")

client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
print(f"✅ System Ready. Using Model: {MODEL_NAME}")

In [ ]:
# @title 2. Agent 1: The Scout (Search)

def parse_search_results(context: CallbackContext):
    """Extracts URL from JSON and saves to state."""
    raw = context.state.get("search_result", "")
    # Robust JSON extraction regex
    match = re.search(r'```json\s*(.*?)\s*```', raw, re.DOTALL)
    json_str = match.group(1) if match else raw.strip()
    
    try:
        data = json.loads(json_str)
        context.state["company_name"] = data.get("company")
        context.state["pdf_url"] = data.get("url")
        context.state["status"] = data.get("status")
        print(f"\n🎯 [Scout] Found URL for {data.get('company')}")
    except:
        print("❌ [Scout] Failed to parse JSON.")

InitialSearchAgent = LlmAgent(
    name="InitialSearchAgent",
    model=Gemini(model=MODEL_NAME),
    tools=[google_search],
    output_key="search_result",
    after_agent_callback=parse_search_results,
    instruction="""
    Find the *direct PDF URL* of the latest Annual Report (2024/2023).
    Query Format: "{company} investor relations annual report 2024 filetype:pdf"
    
    OUTPUT JSON ONLY:
    {"status": "FOUND", "url": "https://...pdf", "company": "..."}
    """
)

In [ ]:
# @title 3. Agent 2: The Analyst (File Processing)

# --- A. CALLBACK: DOWNLOAD & UPLOAD ---
def setup_file_tools(context: CallbackContext):
    state = context.state
    if state.get("file_store_name"):
        print("✅ File already loaded.")
        return

    url = state.get("pdf_url")
    if not url: return

    print(f"\n⬇️ [Analyst] Downloading Report...")
    try:
        # Download
        resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=30)
        fname = "report.pdf"
        with open(fname, 'wb') as f: f.write(resp.content)
        
        # Upload (Gemini File API)
        print("☁️ [Analyst] Uploading to Gemini...")
        store = client.file_search_stores.create(config=types.CreateFileSearchStoreConfig(display_name='Report_Store'))
        op = client.file_search_stores.upload_to_file_search_store(
            file_search_store_name=store.name,
            file=fname,
            config=types.UploadToFileSearchStoreConfig(display_name='Annual Report')
        )
        while not (op := client.operations.get(op)).done: time.sleep(2)
        
        state["file_store_name"] = store.name
        print(f"✅ Store Ready: {store.name}")
    except Exception as e:
        print(f"❌ Upload Failed: {e}")

# --- B. TOOL: RATE-LIMITED QUERY ---
def ask_annual_report(store_name: str, question: str) -> str:
    """Queries the report. Includes mandatory 5s sleep to prevent 429 errors."""
    # 🛑 HARD RATE LIMIT: Force 5s pause per call
    time.sleep(5)
    
    print(f"🔎 [Tool] Querying: {question[:50]}...")
    try:
        # Automatic Retry Logic
        for attempt in range(3):
            try:
                res = client.models.generate_content(
                    model=MODEL_NAME,
                    contents=question,
                    config=types.GenerateContentConfig(
                        tools=[types.Tool(file_search=types.FileSearch(file_search_store_names=[store_name]))]
                    )
                )
                return res.text
            except Exception as e:
                if "429" in str(e): 
                    time.sleep(10 * (attempt + 1)) # Backoff 10s, 20s...
                else: raise e
    except Exception as e:
        return f"Tool Error: {e}"

# --- C. AGENT DEFINITION ---
analyst_agent = LlmAgent(
    name="FileAnalystAgent",
    model=Gemini(model=MODEL_NAME),
    output_key="company_analysis",
    tools=[FunctionTool(ask_annual_report)],
    before_agent_callback=setup_file_tools,
    instruction="""
    You are a Senior Analyst. Use `ask_annual_report` (with the `store_name` from context) to analyze the file.

    YOUR TASK:
    1. Call the tool multiple times to gather deep details on these 6 sections:
       - Company Overview
       - Financial Performance (Revenue, Net Income, Margins)
       - Strategic Execution
       - Products & Services
       - Governance & Risk
       - Future Plans

    2. INTERNAL COMPILATION: 
       Do not output partial data. Gather everything, then output **ONE single, comprehensive Markdown report** containing all 6 sections.
    """
)

In [ ]:
# @title 4. Agents 3 & 4: The Gap Loop

# Force a 15s break between Analyst and GapFinder to refill quota
def cooldown(context: CallbackContext):
    print("\n⏳ [System] Cooling down (15s)...")
    time.sleep(15)

def exit_loop(context: ToolContext):
    print("🏁 [GapFinder] Report Approved.")
    context.actions.escalate = True
    return "Terminating"

gap_finder = LlmAgent(
    name="GapFinderAgent",
    model=Gemini(model=MODEL_NAME),
    tools=[FunctionTool(exit_loop)],
    before_agent_callback=cooldown,
    instruction="""
    Review the `company_analysis`.
    If all 6 sections (Overview, Financials, Strategy, Products, Gov/Risk, Future) are detailed -> Call `exit_loop`.
    Else -> Output list of MISSING items.
    """
)

supplemental = LlmAgent(
    name="SupplementalAgent",
    model=Gemini(model=MODEL_NAME),
    tools=[google_search],
    output_key="company_analysis",
    instruction="""
    Find missing data using Google Search.
    Merge it into `company_analysis` and output the FULL updated report.
    """
)

gap_loop = LoopAgent(
    name="GapClosureLoop",
    sub_agents=[gap_finder, supplemental],
    max_iterations=3
)

In [ ]:
# @title 5. Run Pipeline

pipeline = SequentialAgent(
    name="InvestmentPipeline",
    sub_agents=[InitialSearchAgent, analyst_agent, gap_loop]
)

runner = InMemoryRunner(agent=pipeline)

target = "Etisalat"
print(f"🚀 Starting Analysis for: {target}")

async for event in runner.run_async(f"Analyze {target}"):
    if event.turn_complete:
        print(f"\n🤖 {event.author} Finished Step.")
        # print(event.content.parts[0].text) # Uncomment for full logs